In [69]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from konlpy.tag import Okt


In [70]:
import os
os.environ['JAVA_HOME'] = 'C:\\Program Files\\Java\\jdk-21'  # 여기에 자신의 JDK 설치 경로를 입력하세요


In [71]:
cafe = pd.read_csv("./data/gangnam_cafe.csv")
rest = pd.read_csv("./data/gangnam_restaurant.csv")
hosp = pd.read_csv("./data/g_hospital_all2.csv") 

In [72]:
cafe.info()
print(len(cafe))

rest.info()
print(len(rest))

hosp.info()
print(len(hosp))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    107 non-null    object
 1   type    107 non-null    object
 2   review  107 non-null    object
dtypes: object(3)
memory usage: 2.6+ KB
107
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    100 non-null    object
 1   type    100 non-null    object
 2   review  100 non-null    object
dtypes: object(3)
memory usage: 2.5+ KB
100
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  100 non-null    int64 
 1   name        100 non-null    object
 2   type        100 non-null    object
 3   review      100 non-null    object
dt

In [73]:
hosp = hosp.drop('Unnamed: 0', axis=1)

In [74]:
hosp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    100 non-null    object
 1   type    100 non-null    object
 2   review  100 non-null    object
dtypes: object(3)
memory usage: 2.5+ KB


In [75]:
# 불러온 데이터의 값이 비어 있는지 확인
print('Null값이 있는지 확인합니다.',cafe.isnull().values.any()) # Null 값이 존재하는지 확인 (False=정상)
print('')
cafe = cafe.dropna(how = 'any') # Null 값이 존재하는 행 제거
print("매장명:", cafe['name'].nunique())
print('')
cafe = cafe.drop_duplicates()   # 중복 데이터 프레임 제거 
cafe = cafe.reset_index(drop=True) # 데이터 프레임 재색인
print('중복 및, NULL값을 제거한 후, 다시 NULL값을 확인 합니다.', cafe.isnull().values.any()) # Null 값이 존재하는지 확인 (False=정상)
print('')
cafe

Null값이 있는지 확인합니다. False

매장명: 99

중복 및, NULL값을 제거한 후, 다시 NULL값을 확인 합니다. False



,name,type,review
0,한과와락,카페,여기 개성주악 맛있어욤 벌써 2번째입니다\n강남에 이렇게 이쁜 카페 있는줄 몰랐다 ...
1,레뽀드라라 강남점,카페,비엔나 커피 맛있어요\n유명한 스콘집이라고 왔어요 분위기도좋고 스콘도 맛있네요 특이...
2,한과와락,카페,여기 개성주악 맛있어욤 벌써 2번째입니다\n강남에 이렇게 이쁜 카페 있는줄 몰랐다 ...
3,스템커피 강남점,카페,디저트 종류도 많고 카페도 깨끗해서 좋아요:)\n라떼 넘 맛있어엽! 전반적으로 달달...
4,알베르,카페,대화\n복잡한 강남 한복판에 이런 공간이 있다니!!\n공간이 넓어서 여유있고 편해요...
...,...,...,...
97,빌리엔젤 강남358타워점,카페,"강남역에 3곳중 이 곳으로 오게 되었는데 매우 별로네요, 그리고 네이버 이용시간으로..."
98,잠바주스 서초삼성타운점,카페,굿\n매장에 들어가자마자 과일향이 화악~~~!\n굿\n직원분이 다소 친절하지 않음\...
99,어스마카롱,카페,베이글 아메 세트 저렴해서 좋아요.\n마카롱 귀엽고 맛있어요\n꽈배기도 맛있고 베이...
100,시티갤러리카페 본점,카페,"굿\n베이글이랑 커피 맛있어용\n아이스크림라떼, 단호박크로플, 리코타시티샐러드 이렇..."


In [76]:
# 불러온 데이터의 값이 비어 있는지 확인
print('Null값이 있는지 확인합니다.',rest.isnull().values.any()) # Null 값이 존재하는지 확인 (False=정상)
print('')
rest = rest.dropna(how = 'any') # Null 값이 존재하는 행 제거
print("매장명:", rest['name'].nunique())
print('')
rest = rest.drop_duplicates()   # 중복 데이터 프레임 제거 
rest = rest.reset_index(drop=True) # 데이터 프레임 재색인
print('중복 및, NULL값을 제거한 후, 다시 NULL값을 확인 합니다.', rest.isnull().values.any()) # Null 값이 존재하는지 확인 (False=정상)
print('')
rest

Null값이 있는지 확인합니다. False

매장명: 100

중복 및, NULL값을 제거한 후, 다시 NULL값을 확인 합니다. False



,name,type,review
0,시선 강남점,식당,메뉴들이 다 이쁘고 분위기도 좋아요!! 특히 창가자리 밖이 훤히 보이고 너무 좋슴당...
1,을지다락 강남,식당,파스타랑 사이드 메뉴 감튀를 시켜서 맛나게 먹었습니다~♡♡ 매콤 파스타랑 로제 파스...
2,후추포인트,식당,음.. 결론만 말하면 제가 20대 중후반까지는 맛있게 먹었을 것 같은데 차라리 3-...
3,이자카야 열,식당,"인당 35,000원 이상 주문하면 콜키지 무한프리 ~ 일본에서 업어온 닷사이23 맛..."
4,떡도리탕 강남본점,식당,맛있고 저희는 오픈런으로 4시 58분에 들어갔는데 이미 60퍼센트는 식사중이시더라구...
...,...,...,...
95,라그릴리아 SPC스퀘어점,식당,일정이 많아 점심을 못먹고 저녁일정 전에 혼자 망설이다 들렸는데 (고기가 너무 땡겼...
96,수플레 팬케이크 소과당 본점,식당,👍\n굿\n맛있어요!!\n너무너무 맛있는 푸딩빙수에요 !!! \n강남에서도 푸딩빙수...
97,시골야채된장,식당,건강한 집밥 먹고 싶어서 갔어요. 청국장은 2인분부터 주문 가능해요~ 계란찜도 주셔...
98,마녀주방 강남점,식당,".\n이탈리안 레스토랑 맛집으로, 데이트코스에 완전 추천합니다 ㅎㅎ\n엄청 특이했던..."


In [77]:
# 불러온 데이터의 값이 비어 있는지 확인
print('Null값이 있는지 확인합니다.',hosp.isnull().values.any()) # Null 값이 존재하는지 확인 (False=정상)
print('')
hosp = hosp.dropna(how = 'any') # Null 값이 존재하는 행 제거
print("매장명:", hosp['name'].nunique())
print('')
hosp = hosp.drop_duplicates()   # 중복 데이터 프레임 제거 
hosp = hosp.reset_index(drop=True) # 데이터 프레임 재색인
print('중복 및, NULL값을 제거한 후, 다시 NULL값을 확인 합니다.', hosp.isnull().values.any()) # Null 값이 존재하는지 확인 (False=정상)
print('')
hosp

Null값이 있는지 확인합니다. False

매장명: 95

중복 및, NULL값을 제거한 후, 다시 NULL값을 확인 합니다. False



,name,type,review
0,경희에스아이(SI)한의원,한의원,평소에 소화도 잘 안되고 두통도 자주 있는편인데 우연히 검색하다가 삼실근처에 경희에...
1,생활건강한의원,한의원,혼자 다이어트 하기 힘들어서 여러가지 방법을 시도하던중에 여기 한약을 한번 먹어봤는...
2,보명한의원,한의원,사마귀가 생겨서 검색중 사마귀 전문 한의원 처음 봐서 와보았습니다! 오랫동안 한자리...
3,로아한의원 강남점,한의원,\n하이코 알아보던 중에 로아한의원 비너스코를 알게됬는데 하기를 너무 잘한거 같아요...
4,자연과한의원 강남점,한의원,와 여지껏 다녀본 병원중에 의사쌤 최고친절😊👍👍👍👍 진료받으러 왔다가 너무 친절하셔...
...,...,...,...
92,바른이비인후과의원,이비인후과,의사님이 친절하십니다\n친절해요\n원장님 친절하시고 목이 안좋아서 한달사이 두번이나...
93,김성완이비인후과의원,이비인후과,언제나 친절한 의사 선생님. 치료도 잘해주십니다\n좋아요\n좋아요 \n여의사쌤 친절...
94,연세이비인후과의원,이비인후과,😶\n젊고 친절한 의사 선생님. 필요한 동네 병원\n뵹원\n굳\n굳\n좋아요\n친절...
95,연이비인후과의원,이비인후과,굿\n직원분이 추천해서 갔는데\n정말 친절하시고 상세한 진료에 감사함을 느낍니다\n...


In [78]:
all = pd.concat([cafe, rest, hosp])

In [79]:
print(all.head(300))

           name   type                                             review
0          한과와락     카페  여기 개성주악 맛있어욤 벌써 2번째입니다\n강남에 이렇게 이쁜 카페 있는줄 몰랐다 ...
1     레뽀드라라 강남점     카페  비엔나 커피 맛있어요\n유명한 스콘집이라고 왔어요 분위기도좋고 스콘도 맛있네요 특이...
2          한과와락     카페  여기 개성주악 맛있어욤 벌써 2번째입니다\n강남에 이렇게 이쁜 카페 있는줄 몰랐다 ...
3      스템커피 강남점     카페  디저트 종류도 많고 카페도 깨끗해서 좋아요:)\n라떼 넘 맛있어엽! 전반적으로 달달...
4           알베르     카페  대화\n복잡한 강남 한복판에 이런 공간이 있다니!!\n공간이 넓어서 여유있고 편해요...
..          ...    ...                                                ...
92    바른이비인후과의원  이비인후과  의사님이 친절하십니다\n친절해요\n원장님 친절하시고 목이 안좋아서 한달사이 두번이나...
93   김성완이비인후과의원  이비인후과  언제나 친절한 의사 선생님. 치료도 잘해주십니다\n좋아요\n좋아요 \n여의사쌤 친절...
94    연세이비인후과의원  이비인후과  😶\n젊고 친절한 의사 선생님. 필요한 동네 병원\n뵹원\n굳\n굳\n좋아요\n친절...
95     연이비인후과의원  이비인후과  굿\n직원분이 추천해서 갔는데\n정말 친절하시고 상세한 진료에 감사함을 느낍니다\n...
96  연세위드이비인후과의원  이비인후과  친절\n이빈후과 완전 잘해요\n친절하고 좋아요\n교대역에 병원들이 다 효과가 없어서...

[299 rows x 3 columns]


In [82]:
all['name'].nunique(), all['review'].nunique()

(281, 292)

In [83]:
all.drop_duplicates(subset='name', keep='first', inplace=True, ignore_index=True)
print(all)

            name   type                                             review
0           한과와락     카페  여기 개성주악 맛있어욤 벌써 2번째입니다\n강남에 이렇게 이쁜 카페 있는줄 몰랐다 ...
1      레뽀드라라 강남점     카페  비엔나 커피 맛있어요\n유명한 스콘집이라고 왔어요 분위기도좋고 스콘도 맛있네요 특이...
2       스템커피 강남점     카페  디저트 종류도 많고 카페도 깨끗해서 좋아요:)\n라떼 넘 맛있어엽! 전반적으로 달달...
3            알베르     카페  대화\n복잡한 강남 한복판에 이런 공간이 있다니!!\n공간이 넓어서 여유있고 편해요...
4           호랑가시     카페  맛있어요. ㅎㅎㅎ \n일단 대형카페여서 너무좋고 빵들도 특이한게 많아요 맛도 좋고 ...
..           ...    ...                                                ...
276    바른이비인후과의원  이비인후과  의사님이 친절하십니다\n친절해요\n원장님 친절하시고 목이 안좋아서 한달사이 두번이나...
277   김성완이비인후과의원  이비인후과  언제나 친절한 의사 선생님. 치료도 잘해주십니다\n좋아요\n좋아요 \n여의사쌤 친절...
278    연세이비인후과의원  이비인후과  😶\n젊고 친절한 의사 선생님. 필요한 동네 병원\n뵹원\n굳\n굳\n좋아요\n친절...
279     연이비인후과의원  이비인후과  굿\n직원분이 추천해서 갔는데\n정말 친절하시고 상세한 진료에 감사함을 느낍니다\n...
280  연세위드이비인후과의원  이비인후과  친절\n이빈후과 완전 잘해요\n친절하고 좋아요\n교대역에 병원들이 다 효과가 없어서...

[281 rows x 3 columns]


In [84]:
all['name'].nunique(), all['review'].nunique()

(281, 281)

In [80]:
# 불러온 데이터의 값이 비어 있는지 확인
print('Null값이 있는지 확인합니다.',all.isnull().values.any()) # Null 값이 존재하는지 확인 (False=정상)
print('')
all = all.dropna(how = 'any') # Null 값이 존재하는 행 제거
print("매장명:", all['name'].nunique())
print('')
all = all.drop_duplicates()   # 중복 데이터 프레임 제거 
all = all.reset_index(drop=True) # 데이터 프레임 재색인
print('중복 및, NULL값을 제거한 후, 다시 NULL값을 확인 합니다.', all.isnull().values.any()) # Null 값이 존재하는지 확인 (False=정상)
print('')
all

Null값이 있는지 확인합니다. False

매장명: 281

중복 및, NULL값을 제거한 후, 다시 NULL값을 확인 합니다. False



,name,type,review
0,한과와락,카페,여기 개성주악 맛있어욤 벌써 2번째입니다\n강남에 이렇게 이쁜 카페 있는줄 몰랐다 ...
1,레뽀드라라 강남점,카페,비엔나 커피 맛있어요\n유명한 스콘집이라고 왔어요 분위기도좋고 스콘도 맛있네요 특이...
2,한과와락,카페,여기 개성주악 맛있어욤 벌써 2번째입니다\n강남에 이렇게 이쁜 카페 있는줄 몰랐다 ...
3,스템커피 강남점,카페,디저트 종류도 많고 카페도 깨끗해서 좋아요:)\n라떼 넘 맛있어엽! 전반적으로 달달...
4,알베르,카페,대화\n복잡한 강남 한복판에 이런 공간이 있다니!!\n공간이 넓어서 여유있고 편해요...
...,...,...,...
294,바른이비인후과의원,이비인후과,의사님이 친절하십니다\n친절해요\n원장님 친절하시고 목이 안좋아서 한달사이 두번이나...
295,김성완이비인후과의원,이비인후과,언제나 친절한 의사 선생님. 치료도 잘해주십니다\n좋아요\n좋아요 \n여의사쌤 친절...
296,연세이비인후과의원,이비인후과,😶\n젊고 친절한 의사 선생님. 필요한 동네 병원\n뵹원\n굳\n굳\n좋아요\n친절...
297,연이비인후과의원,이비인후과,굿\n직원분이 추천해서 갔는데\n정말 친절하시고 상세한 진료에 감사함을 느낍니다\n...


## 토크나이징 및 전처리

In [89]:
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', } 


def clean(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text.strip()


import re


def clean_str(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '[^\w\s\n]'         # 특수기호제거
    text = re.sub(pattern=pattern, repl='', string=text)
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','', string=text)
    text = re.sub('\n', ' ', string=text)
    text = text.replace('  ', ' ')
    return text 

In [90]:
all['review'] = all['review'].apply(clean_str)
all['review']

0      여기 개성주악 맛있어욤 벌써 2번째입니다 강남에 이렇게 이쁜 카페 있는줄 몰랐다 했...
1      비엔나 커피 맛있어요 유명한 스콘집이라고 왔어요 분위기도좋고 스콘도 맛있네요 특이한...
2      디저트 종류도 많고 카페도 깨끗해서 좋아요 라떼 넘 맛있어엽 전반적으로 달달한 맛이...
3      대화 복잡한 강남 한복판에 이런 공간이 있다니 공간이 넓어서 여유있고 편해요 아메리...
4      맛있어요  일단 대형카페여서 너무좋고 빵들도 특이한게 많아요 맛도 좋고 커피맛도 아...
                             ...                        
276    의사님이 친절하십니다 친절해요 원장님 친절하시고 목이 안좋아서 한달사이 두번이나 갔...
277    언제나 친절한 의사 선생님 치료도 잘해주십니다 좋아요 좋아요 여의사쌤 친절하세요 좋...
278     젊고 친절한 의사 선생님 필요한 동네 병원 뵹원 굳 굳 좋아요 친절해서 좋아요 좋...
279    굿 직원분이 추천해서 갔는데 정말 친절하시고 상세한 진료에 감사함을 느낍니다 굿 꼼...
280    친절 이빈후과 완전 잘해요 친절하고 좋아요 교대역에 병원들이 다 효과가 없어서 마지...
Name: review, Length: 281, dtype: object

In [92]:
# 원하는 단어 사전에 추가할 수 있는 버전
from ckonlpy.tag import Twitter

twitter = Twitter()

twitter.add_dictionary(['규칙적','전문적','맞춤형','원장님'],'Noun')

# raw_pos_tagged = twitter.pos(articles_1,norm=True, stem=True)
# raw_morphs_tagged = twitter.morphs(articles_1,norm=True, stem=True)
# print(raw_pos_tagged)
# print(raw_morphs_tagged)

raw_pos_tagged = [] # 훈련할 코퍼스

for sentence in (all['review']):
  temp = twitter.pos(sentence, stem=True, norm=True) # 각 문장에 대해 형태소 분리가 일어난다.
  raw_pos_tagged.append(temp)

raw_morphs_tagged = [] # 훈련할 코퍼스

for sentence in (all['review']):
  temp = twitter.morphs(sentence, stem=True, norm=True) # 각 문장에 대해 형태소 분리가 일어난다.
  raw_morphs_tagged.append(temp)

c:\Users\figure.2\AppData\Local\Programs\Python\Python311\Lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [124]:
# 원하는 단어 사전에 추가할 수 있는 버전
from ckonlpy.tag import Twitter
from tqdm import tqdm_notebook

twitter = Twitter()

twitter.add_dictionary(['규칙적','전문적','맞춤형','원장님'],'Noun')

raw_pos_tagged = [] # 훈련할 코퍼스

for sentence in tqdm_notebook(df_r['review']):
  temp = twitter.pos(sentence, stem=True, norm=True) # 각 문장에 대해 형태소 분리가 일어난다.
  raw_pos_tagged.append(temp)

raw_morphs_tagged = [] # 훈련할 코퍼스

for sentence in tqdm_notebook(df_r['review']):
  temp = twitter.morphs(sentence, stem=True, norm=True) # 각 문장에 대해 형태소 분리가 일어난다.
  raw_morphs_tagged.append(temp)

c:\Users\figure.2\AppData\Local\Programs\Python\Python311\Lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


NameError: name 'df_r' is not defined

In [93]:
print(len(raw_morphs_tagged))
print(len(raw_morphs_tagged[0]))
print(raw_morphs_tagged[:3])
print(raw_pos_tagged[:1])

281
2042
[['여기', '개성', '주악', '맛있다', '욤', '벌써', '2', '번', '째', '이다', '강남', '에', '이렇게', '이쁘다', '카페', '있다', '모르다', '하다', '더니', '지난달', '에', '오픈', '하다', '경치', '도', '좋다', '차', '향', '도', '좋다', '인테리어', '도', '너무', '잘', '하다', '어요', '다만', '공간', '이', '막', '넓다', '직', '넓다', '직하', '진', '않다', '계단', '을', '좀', '오르다', '풍경', '바람', '불면', '찰랑', '찰랑', '하다', '소리', '내다', '있다', '차다', '좋다', '생각', '이', '들다', '어요', '기분', '좋다', '식후차', '하고', '가다', '다음', '엔', '여자친구', '데려오다', '정성', '스럽게', '주다', '차', '맛있다', '약과', '는', '너무', '달다', '공장', '형', '약과', '와는', '다른', '개성', '약과', '가', '달', '지', '않다', '너무', '좋다', '다양', '한', '맛', '의', '양갱', '도', '수제', '라서', '그렇다', '부드럽다', '입', '에', '딱', '맞다', '어요', '다시', '방문', '해서', '주악', '도', '먹다', '어보', '려고', '해', '요', '주악', '메뉴', '들다', '다', '맛있다', '다른', '메뉴', '들다', '다소', '평범', '하다', '느껴지다', '기본', '주악', '사르다', '하다', '번', '더', '가다', '한과', '디저트', '너무', '맛있다', '어요', '여기', '진짜', '존맛', '이에요', '근데', '주말', '에', '갓', '더니', '자리', '하나', '두', '없다', '어서', '포장', '하다', '어요', '가게', '안도', '예쁘다', '음료', '도', '예쁘다

In [110]:
stop = open('./data/cleaned_stopword_update2.txt','r', encoding='utf-8')
stopwords = stop.read().splitlines()
stopwords

['가',
 '가까스로',
 '가령',
 '각',
 '각각',
 '각자',
 '각종',
 '갖고말하자면',
 '같다',
 '같이',
 '개의치않고',
 '거니와',
 '거바',
 '거의',
 '것',
 '것과 같이',
 '것들',
 '게다가',
 '게우다',
 '겨우',
 '견지에서',
 '결과에 이르다',
 '결국',
 '결론을 낼 수 있다',
 '겸사겸사',
 '고려하면',
 '고로',
 '곧',
 '공동으로',
 '과',
 '과연',
 '관계가 있다',
 '관계없이',
 '관련이 있다',
 '관하여',
 '관한',
 '관해서는',
 '구',
 '구체적으로',
 '구토하다',
 '그',
 '그들',
 '그때',
 '그래',
 '그래도',
 '그래서',
 '그러나',
 '그러니',
 '그러니까',
 '그러면',
 '그러므로',
 '그러한즉',
 '그런 까닭에',
 '그런데',
 '그런즉',
 '그럼',
 '그럼에도 불구하고',
 '그렇게 함으로써',
 '그렇지',
 '그렇지 않다면',
 '그렇지 않으면',
 '그렇지만',
 '그렇지않으면',
 '그리고',
 '그리하여',
 '그만이다',
 '그에 따르는',
 '그위에',
 '그저',
 '그중에서',
 '그치지 않다',
 '근거로',
 '근거하여',
 '기대여',
 '기점으로',
 '기준으로',
 '기타',
 '까닭으로',
 '까악',
 '까지',
 '까지 미치다',
 '까지도',
 '꽈당',
 '끙끙',
 '끼익',
 '나',
 '나머지는',
 '남들',
 '남짓',
 '너',
 '너희',
 '너희들',
 '네',
 '넷',
 '년',
 '논하지 않다',
 '놀라다',
 '누가 알겠는가',
 '누구',
 '다른',
 '다른 방면으로',
 '다만',
 '다섯',
 '다소',
 '다수',
 '다시 말하자면',
 '다시말하면',
 '다음',
 '다음에',
 '다음으로',
 '단지',
 '답다',
 '당신',
 '당장',
 '대로 하다',
 '대하면',
 '대하여',
 '대해 말하자면',
 '대해서',
 '댕그',


## 'Verb','Adjective','Noun' 만 추출 할떄 

In [111]:
raw_pos = []

for setence in raw_pos_tagged:
    pos_pick = []
    for word, tag in setence:
        if tag in ['Verb','Adjective','Noun'] and word not in stopwords:
            pos_pick.append(word)
    raw_pos.append(pos_pick)

print(len(raw_pos))
print(len(raw_pos[0]))
print(raw_pos[:3])


281
1283
[['개성', '주악', '맛있다', '욤', '벌써', '번', '강남', '이쁘다', '카페', '모르다', '더니', '지난달', '오픈', '경치', '좋다', '차', '향', '좋다', '인테리어', '공간', '막', '넓다', '직', '넓다', '직하', '진', '계단', '풍경', '바람', '불면', '찰랑', '찰랑', '소리', '내다', '차다', '좋다', '생각', '들다', '기분', '좋다', '식후차', '가다', '여자친구', '데려오다', '정성', '주다', '차', '맛있다', '약과', '달다', '공장', '형', '약과', '개성', '약과', '달', '좋다', '다양', '맛', '양갱', '수제', '라서', '그렇다', '부드럽다', '입', '맞다', '다시', '방문', '주악', '먹다', '어보', '려고', '해', '주악', '메뉴', '들다', '맛있다', '메뉴', '들다', '평범', '느껴지다', '기본', '주악', '사르다', '번', '더', '가다', '한과', '디저트', '맛있다', '진짜', '존맛', '주말', '갓', '더니', '자리', '없다', '어서', '포장', '가게', '안도', '예쁘다', '음료', '예쁘다', '잔', '담다', '주시', '데', '넘다', '아쉽다', '담', '평일', '방문', '야겟어용', '주악', '맛있다', '주악', '맛있다', '좋다', '개성', '주악', '인식', '바뀌다', '곳', '이전', '백화점', '먹다', '기름', '냄새', '많다', '이나고', '달기', '먹다', '쫄깃달콤', '고소', '세', '박자', '넘다', '자다', '맞다', '작다', '해도', '가격', '사악', '좋다', '따다', '데우다', '제공', '주다', '더', '좋다', '알다', '곳', '개성', '주악', '맛', '잇다', '거', '생', '긴지', '안', '개성', '주악', '안',

In [112]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(raw_pos)
print(tokenizer.word_index)

{'좋다': 1, '맛있다': 2, '친절': 3, '먹다': 4, '자다': 5, '받다': 6, '들다': 7, '오다': 8, '맛': 9, '어서': 10, '가다': 11, '보다': 12, '해주': 13, '방문': 14, '굿': 15, '많다': 16, '원장님': 17, '진료': 18, '곳': 19, '없다': 20, '직원': 21, '치료': 22, '분': 23, '진짜': 24, '분위기': 25, '정말': 26, '커피': 27, '넘다': 28, '친': 29, '선생님': 30, '매장': 31, '해주시': 32, '감사': 33, '추천': 34, '설명': 35, '상담': 36, '절하': 37, '셔': 38, '가격': 39, '음식': 40, '시술': 41, '넓다': 42, '만족': 43, '수': 44, '더': 45, '병원': 46, '강남': 47, '의사': 48, '맛집': 49, '나오다': 50, '처음': 51, '사람': 52, '고기': 53, '강남역': 54, '끄다': 55, '아프다': 56, '맞다': 57, '주시': 58, '고': 59, '싶다': 60, '데': 61, '거': 62, '안': 63, '생각': 64, '카페': 65, '효과': 66, '다가': 67, '다니다': 68, '메뉴': 69, '비': 70, '예약': 71, '면서': 72, '빵': 73, '깔다': 74, '인테리어': 75, '음료': 76, '최고': 77, '크다': 78, '주문': 79, '주다': 80, '편하다': 81, '전': 82, '느낌': 83, '괜찮다': 84, '찾다': 85, '적': 86, '갈다': 87, '항상': 88, '디저트': 89, '예쁘다': 90, '마다': 91, '기분': 92, '알다': 93, '아주': 94, '치과': 95, '정도': 96, '가성': 97, '그렇다': 98, '자주': 99, '안주': 100, '완전': 1

## 'Adjective','Noun' 만 추출 

In [115]:
raw_pos = []

for setence in raw_pos_tagged:
    pos_pick = []
    for word, tag in setence:
        if tag in ['Adjective','Noun'] and word not in stopwords:
            pos_pick.append(word)
    raw_pos.append(pos_pick)

print(len(raw_pos))
print(len(raw_pos[0]))
print(raw_pos[:3])


281
1079
[['개성', '주악', '맛있다', '욤', '벌써', '번', '강남', '이쁘다', '카페', '더니', '지난달', '오픈', '경치', '좋다', '차', '향', '좋다', '인테리어', '공간', '막', '넓다', '직', '넓다', '직하', '진', '계단', '풍경', '바람', '불면', '찰랑', '찰랑', '소리', '좋다', '생각', '기분', '좋다', '식후차', '여자친구', '정성', '차', '맛있다', '약과', '공장', '형', '약과', '개성', '약과', '달', '좋다', '다양', '맛', '양갱', '수제', '라서', '그렇다', '부드럽다', '입', '다시', '방문', '주악', '어보', '려고', '해', '주악', '메뉴', '맛있다', '메뉴', '평범', '기본', '주악', '번', '더', '한과', '디저트', '맛있다', '진짜', '존맛', '주말', '갓', '더니', '자리', '없다', '어서', '포장', '가게', '안도', '예쁘다', '음료', '예쁘다', '잔', '주시', '데', '아쉽다', '담', '평일', '방문', '야겟어용', '주악', '맛있다', '주악', '맛있다', '좋다', '개성', '주악', '인식', '곳', '이전', '백화점', '기름', '냄새', '많다', '이나고', '달기', '쫄깃달콤', '고소', '세', '박자', '작다', '해도', '가격', '사악', '좋다', '제공', '더', '좋다', '곳', '개성', '주악', '맛', '거', '생', '긴지', '안', '개성', '주악', '안', '봄', '가보', '처음', '어보', '개성', '주악', '찹쌀떡', '맛', '더', '쫀득', '바싹', '덕', '느낌', '었다', '달', '좋다', '원래', '양갱', '별로', '안', '좋다', '데', '맛있다', '이름', '얼그레이', '어쩌구', '디저트', '취향', '었다', '한

In [117]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(raw_pos)
print(tokenizer.word_index)

{'좋다': 1, '맛있다': 2, '친절': 3, '맛': 4, '어서': 5, '해주': 6, '방문': 7, '굿': 8, '많다': 9, '원장님': 10, '진료': 11, '곳': 12, '없다': 13, '직원': 14, '치료': 15, '분': 16, '진짜': 17, '분위기': 18, '정말': 19, '커피': 20, '친': 21, '선생님': 22, '매장': 23, '해주시': 24, '감사': 25, '추천': 26, '설명': 27, '상담': 28, '절하': 29, '셔': 30, '가격': 31, '음식': 32, '시술': 33, '넓다': 34, '만족': 35, '수': 36, '더': 37, '병원': 38, '강남': 39, '의사': 40, '맛집': 41, '처음': 42, '사람': 43, '고기': 44, '강남역': 45, '아프다': 46, '주시': 47, '고': 48, '데': 49, '거': 50, '안': 51, '생각': 52, '카페': 53, '효과': 54, '다가': 55, '메뉴': 56, '비': 57, '예약': 58, '면서': 59, '빵': 60, '인테리어': 61, '음료': 62, '최고': 63, '주문': 64, '편하다': 65, '전': 66, '느낌': 67, '괜찮다': 68, '적': 69, '항상': 70, '디저트': 71, '예쁘다': 72, '마다': 73, '기분': 74, '아주': 75, '치과': 76, '정도': 77, '가성': 78, '그렇다': 79, '자주': 80, '안주': 81, '완전': 82, '다양': 83, '비싸다': 84, '재': 85, '자리': 86, '약': 87, '하니': 88, '중': 89, '양': 90, '스럽다': 91, '간호사': 92, '교정': 93, '말': 94, '후': 95, '종류': 96, '굳다': 97, '집': 98, '술': 99, '피부': 100, '용': 101, '꼭':

In [122]:
print(tokenizer.word_index)

{'좋다': 1, '맛있다': 2, '친절': 3, '맛': 4, '어서': 5, '해주': 6, '방문': 7, '굿': 8, '많다': 9, '원장님': 10, '진료': 11, '곳': 12, '없다': 13, '직원': 14, '치료': 15, '분': 16, '진짜': 17, '분위기': 18, '정말': 19, '커피': 20, '친': 21, '선생님': 22, '매장': 23, '해주시': 24, '감사': 25, '추천': 26, '설명': 27, '상담': 28, '절하': 29, '셔': 30, '가격': 31, '음식': 32, '시술': 33, '넓다': 34, '만족': 35, '수': 36, '더': 37, '병원': 38, '강남': 39, '의사': 40, '맛집': 41, '처음': 42, '사람': 43, '고기': 44, '강남역': 45, '아프다': 46, '주시': 47, '고': 48, '데': 49, '거': 50, '안': 51, '생각': 52, '카페': 53, '효과': 54, '다가': 55, '메뉴': 56, '비': 57, '예약': 58, '면서': 59, '빵': 60, '인테리어': 61, '음료': 62, '최고': 63, '주문': 64, '편하다': 65, '전': 66, '느낌': 67, '괜찮다': 68, '적': 69, '항상': 70, '디저트': 71, '예쁘다': 72, '마다': 73, '기분': 74, '아주': 75, '치과': 76, '정도': 77, '가성': 78, '그렇다': 79, '자주': 80, '안주': 81, '완전': 82, '다양': 83, '비싸다': 84, '재': 85, '자리': 86, '약': 87, '하니': 88, '중': 89, '양': 90, '스럽다': 91, '간호사': 92, '교정': 93, '말': 94, '후': 95, '종류': 96, '굳다': 97, '집': 98, '술': 99, '피부': 100, '용': 101, '꼭':

In [120]:
print(raw_pos)

[['개성', '주악', '맛있다', '욤', '벌써', '번', '강남', '이쁘다', '카페', '더니', '지난달', '오픈', '경치', '좋다', '차', '향', '좋다', '인테리어', '공간', '막', '넓다', '직', '넓다', '직하', '진', '계단', '풍경', '바람', '불면', '찰랑', '찰랑', '소리', '좋다', '생각', '기분', '좋다', '식후차', '여자친구', '정성', '차', '맛있다', '약과', '공장', '형', '약과', '개성', '약과', '달', '좋다', '다양', '맛', '양갱', '수제', '라서', '그렇다', '부드럽다', '입', '다시', '방문', '주악', '어보', '려고', '해', '주악', '메뉴', '맛있다', '메뉴', '평범', '기본', '주악', '번', '더', '한과', '디저트', '맛있다', '진짜', '존맛', '주말', '갓', '더니', '자리', '없다', '어서', '포장', '가게', '안도', '예쁘다', '음료', '예쁘다', '잔', '주시', '데', '아쉽다', '담', '평일', '방문', '야겟어용', '주악', '맛있다', '주악', '맛있다', '좋다', '개성', '주악', '인식', '곳', '이전', '백화점', '기름', '냄새', '많다', '이나고', '달기', '쫄깃달콤', '고소', '세', '박자', '작다', '해도', '가격', '사악', '좋다', '제공', '더', '좋다', '곳', '개성', '주악', '맛', '거', '생', '긴지', '안', '개성', '주악', '안', '봄', '가보', '처음', '어보', '개성', '주악', '찹쌀떡', '맛', '더', '쫀득', '바싹', '덕', '느낌', '었다', '달', '좋다', '원래', '양갱', '별로', '안', '좋다', '데', '맛있다', '이름', '얼그레이', '어쩌구', '디저트', '취향', '었다', '한국인', '맛있다

In [123]:
#길이가 1이하인 단어는 제거하기
num_list = []
for j in range(len(tokenizer.word_index)):
    temp = []
    for k in range(len(tokenizer.word_index[j])):
        if len(tokenizer.word_index[j][k]) > 1:
            temp.append(tokenizer.word_index[j][k])
    num_list.append(temp)

print(num_list)

KeyError: 0

In [113]:
# threshold 만큼 등장하지 않은 단어들의 비율 확인
def print_freq(threshold):

  total_cnt = len(tokenizer.word_index) # 단어 개수
  rare_cnt  = 0 # 등장 빈도가 threshold보다 작은 단어의 개수를 카운트

  total_freq = 0 # 훈련 데이터 전체 단어 빈도수 총 합. 영화: 3번, 재미: 5번 -> total_freq = 8
  rare_freq  = 0 # 사용된 희귀 단어의 총 합

  for key, value in tokenizer.word_counts.items():
    # key : 단어, value : 빈도수
    total_freq = total_freq + value # 모든 단어의 빈도수 총 합 구하기

    # 희귀단어 개수 및 등장 빈도 계산하기
    if value < threshold :
      rare_cnt += 1
      rare_freq += value

  print("단어 집합의 크기 : {}".format(total_cnt))
  print("등장 빈도가 {} 미만인 희귀 단어의 수 : {}".format(threshold, rare_cnt))
  print("단어 집합에서 희귀 단어의 비율 : {:.3f}%".format((rare_cnt / total_cnt) * 100))
  print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율 : {:.3f}%".format((rare_freq / total_freq) * 100))
  print("="*50)

  return total_cnt, rare_cnt

In [114]:
print_freq(2)
print_freq(3)
print_freq(4)
print_freq(5)

단어 집합의 크기 : 12202
등장 빈도가 2 미만인 희귀 단어의 수 : 5120
단어 집합에서 희귀 단어의 비율 : 41.960%
전체 등장 빈도에서 희귀 단어 등장 빈도 비율 : 2.036%
단어 집합의 크기 : 12202
등장 빈도가 3 미만인 희귀 단어의 수 : 6711
단어 집합에서 희귀 단어의 비율 : 54.999%
전체 등장 빈도에서 희귀 단어 등장 빈도 비율 : 3.301%
단어 집합의 크기 : 12202
등장 빈도가 4 미만인 희귀 단어의 수 : 7633
단어 집합에서 희귀 단어의 비율 : 62.555%
전체 등장 빈도에서 희귀 단어 등장 빈도 비율 : 4.401%
단어 집합의 크기 : 12202
등장 빈도가 5 미만인 희귀 단어의 수 : 8214
단어 집합에서 희귀 단어의 비율 : 67.317%
전체 등장 빈도에서 희귀 단어 등장 빈도 비율 : 5.325%


(12202, 8214)

In [94]:
# 'r'은 read mode를 의미합니다.
with open(DATA_STOP_WORD_FILE_NAME, 'r') as f:
    stopwords = f.readlines()

# 각 줄의 앞뒤 공백 제거
stopwords = [word.strip() for word in stopwords]


NameError: name 'DATA_STOP_WORD_FILE_NAME' is not defined

In [81]:
stopword_vocab = DATA_STOP_WORD_FILE_NAME # 불용어 파일 불러오기
sep = "\n" # 불용어 처리 인자
    
# 불용어 데이터를 가져와 리스트로 변환합니다.
with open(stopword_vocab, encoding = 'utf-8') as f:
    temp1 = []
    for i in f:
        temp1.append(i)

stopword_vocab = []

# 불용어 데이터는 전역변수 stopword_vocab 선언합니다. 
# 구분자에 따라 stopword_vocab에 추가하여 불용어 사전을 구축합니다.
for j in range(len(temp1)):
    temp2 = temp1[j].rstrip(separate)
    stopword_vocab.append(temp2)

NameError: name 'DATA_STOP_WORD_FILE_NAME' is not defined

In [96]:
# 'r'은 read mode를 의미합니다.
with open('./data/stopword_update.txt', 'r', encoding='utf-8') as f:
    stopwords = f.readlines()

# 각 줄의 앞뒤 공백 제거
stopwords = [word.strip() for word in stopwords]


In [109]:
# 'r'은 read mode를 의미합니다.
with open('./data/stopword_update.txt', 'r', encoding='utf-8') as f:
    stopwords = f.readlines()

# 각 줄의 앞뒤 공백 제거
stopwords = [word.strip() for word in stopwords]

# 결측치 확인 및 제거
stopwords = [word for word in stopwords if word]  # 이 코드는 빈 문자열을 제거합니다.

# 중복값 확인 및 제거
stopwords = list(set(stopwords))  # set으로 변환하여 중복을 제거하고, 다시 list로 변환

# 정렬 (선택 사항)
stopwords.sort()

# 새로운 파일에 저장
with open('cleaned_stopword_update2.txt', 'w', encoding='utf-8') as f:
    for stopword in stopwords:
        f.write(stopword + '\n')


In [105]:
print(stopwords[:5])

['가', '가까스로', '가령', '각', '각각']


In [103]:
import pandas as pd

stopwords_series = pd.Series(stopwords)
print(stopwords_series.head(200))

0             가
1          가까스로
2            가령
3             각
4            각각
         ...   
195    바꾸어서 말하면
196    바꾸어서 한다면
197      바꿔 말하면
198          바로
199        바와같이
Length: 200, dtype: object
